In [264]:
import torch
import random, os, pdb
import numpy as np

number_of_positive_examples_x_types=250
number_of_negative_examples_x_types=250
number_of_positive_example_x_partof=250
number_of_negative_example_x_partof=250
number_of_pairs_for_axioms = 1000
zero_distance_threshold = 6
number_of_features = 65
data_training_dir = "/data/data_wuxuan/SII/data/training/"
data_testing_dir = "/data/data_wuxuan/SII/data/testing/"
name = "indoor"


types = {}
with open("/data/data_wuxuan/SII/data/classes.csv", "r") as f:
    cnt = 0
    for line in f.readlines():
        types[line.strip()] = cnt
        cnt += 1
        
if name == "vehicle":
    # uncomment this line for training the vehicle object types
    selected_types_name = np.array(['aeroplane','artifact_wing','body','engine','stern','wheel','bicycle','chain_wheel','handlebar','headlight','saddle','bus','bodywork','door','license_plate','mirror','window','car','motorbike','train','coach','locomotive','boat'])

if name == "indoor":
    # uncomment this line for training the indoor object types
    selected_types_name = np.array(['bottle','body','cap','pottedplant','plant','pot','tvmonitor','screen']) #'chair','sofa','diningtable'

if name == "animal":
    # uncomment this line for training the animal object types
    selected_types_name = np.array(['person','arm','ear','ebrow','foot','hair','hand','mouth','nose','eye','head','leg','neck','torso','cat','tail','bird','animal_wing','beak','sheep','horn','muzzle','cow','dog','horse','hoof'])



selected_types = [types[n] for n in selected_types_name]


def containment_ratios_between_two_bbxes(bb1, bb2):
    bb1_area = (bb1[-2] - bb1[-4]) * (bb1[-1] - bb1[-3])
    bb2_area = (bb2[-2] - bb2[-4]) * (bb2[-1] - bb2[-3])
    w_intersec = max(0,min([bb1[-2], bb2[-2]]) - max([bb1[-4], bb2[-4]]))
    h_intersec = max(0,min([bb1[-1], bb2[-1]]) - max([bb1[-3], bb2[-3]]))
    bb_area_intersection = w_intersec * h_intersec
    return [float(bb_area_intersection)/bb1_area, float(bb_area_intersection)/bb2_area]

def get_data(train_or_test_swritch,max_rows=10000000):
    assert train_or_test_swritch == "train" or train_or_test_swritch == "test"

    # Fetching the data from the file system

    if train_or_test_swritch == "train":
        data_dir = data_training_dir
    if train_or_test_swritch == "test":
        data_dir = data_testing_dir
    data = np.genfromtxt(data_dir+"features.csv",delimiter=",",max_rows=max_rows)
    types_of_data = np.genfromtxt(data_dir + "types.csv", dtype="i", max_rows=max_rows)
    idx_whole_for_data = np.genfromtxt(data_dir+ "partOf.csv",dtype="i",max_rows=max_rows)
    idx_of_cleaned_data = np.where(np.logical_and(
        np.all(data[:, -2:] - data[:, -4:-2] >= zero_distance_threshold, axis=1),
        np.in1d(types_of_data,selected_types)))[0]
    print("deleting", len(data) - len(idx_of_cleaned_data), "small bb out of", data.shape[0], "bb")
    data = data[idx_of_cleaned_data]
    data[:, -4:] /= 500

    # Cleaning data by removing small bounding boxes and recomputing indexes of partof data

    types_of_data = types_of_data[idx_of_cleaned_data]
    idx_whole_for_data = idx_whole_for_data[idx_of_cleaned_data]
    for i in range(len(idx_whole_for_data)):
        if idx_whole_for_data[i] != -1 and idx_whole_for_data[i] in idx_of_cleaned_data:
            idx_whole_for_data[i] = np.where(idx_whole_for_data[i] == idx_of_cleaned_data)[0]
        else:
            idx_whole_for_data[i] = -1

    # Grouping bbs that belong to the same picture

    pics = {} #记录了每张图片对应的bbox，即data的id
    for i in range(len(data)):
        if data[i][0] in pics:
            pics[data[i][0]].append(i)
        else:
            pics[data[i][0]] = [i]

    pairs_of_data = np.array(
        [np.concatenate((data[i][1:], data[j][1:], containment_ratios_between_two_bbxes(data[i], data[j]))) for p in
         pics for i in pics[p] for j in pics[p]])

    pairs_of_bb_idxs = np.array([(i,j) for p in pics for i in pics[p] for j in pics[p]]) #枚举同一张图片里不同objects(bbox) pair

    partOf_of_pair_of_data = np.array([idx_whole_for_data[i] == j for p in pics for i in pics[p] for j in pics[p]])

    return data, pairs_of_data, types_of_data, partOf_of_pair_of_data, pairs_of_bb_idxs, pics


data, pairs_of_data, types_of_data, partOf_of_pairs_of_data, pairs_of_bb_idxs, pics = get_data("train",max_rows=1000000000)
print("individual size: ", data.shape[0], "concept size: ", data.shape[1]-4)

deleting 140540 small bb out of 146217 bb
individual size:  5677 concept size:  61


In [217]:
id2types = {idx:t for t,idx in types.items()}

In [276]:
a = [0,0,0,2]
b = 2
import numpy as np
np.true_divide(a,b)

array([0., 0., 0., 1.])

In [218]:

# computing positive and negative examples for types and partof
idxs_of_positive_examples_of_types = {}
idxs_of_negative_examples_of_types = {}

for type in selected_types:
    idxs_of_positive_examples_of_types[type] = np.where(types_of_data == type)[0]
    idxs_of_negative_examples_of_types[type] = np.where(types_of_data != type)[0]

idxs_of_positive_examples_of_partOf = np.where(partOf_of_pairs_of_data)[0]
idxs_of_negative_examples_of_partOf = np.where(partOf_of_pairs_of_data == False)[0]

existing_types = [t for t in selected_types if idxs_of_positive_examples_of_types[t].size > 0]


In [219]:
# domain definition
import ltn
import csv
from torch.autograd import Variable
default_layers = 10
default_smooth_factor = 0.0000001
default_tnorm = "product"
default_optimizer = "gd"
default_aggregator = "min"
default_positive_fact_penality = 1e-6
default_clauses_aggregator = "min"

device = torch.device("cuda:3")
def get_part_whole_ontology():
    with open('/data/data_wuxuan/SII/data/pascalPartOntology.csv','r') as f:
        ontologyReader = csv.reader(f)
        parts_of_whole = {}
        wholes_of_part = {}
        for row in ontologyReader:
            parts_of_whole[row[0]] = row[1:]
            for t in row[1:]:
                if t in wholes_of_part:
                    wholes_of_part[t].append(row[0])
                else:
                    wholes_of_part[t] = [row[0]]
        for whole in parts_of_whole:
            wholes_of_part[whole] = []
        for part in wholes_of_part:
            if part not in parts_of_whole:
                parts_of_whole[part] = []

    selected_parts_of_whole = {}
    selected_wholes_of_part = {}
    for t in selected_types:
        selected_parts_of_whole[t] = [p for p in parts_of_whole[id2types[t]] if p in selected_types]
        selected_wholes_of_part[t] = [w for w in wholes_of_part[id2types[t]] if w in selected_types]
    return selected_parts_of_whole, selected_wholes_of_part




class Domain:
    def __init__(self,columns, dom_type="float",label=None):
        self.columns = columns
        self.label = label
        self.tensor = torch.tensor([0.0])
        self.parameters = []
        
class Predicate(torch.nn.Module):
    def __init__(self, label, domain, layers=default_layers):
        super().__init__()
        self.label = label
        self.domain = domain
        self.number_of_layers = layers
#         self.W = Variable(torch.randn((layers, self.domain.columns, self.domain.columns), names = ("W_"+label+"_0", "W_"+label+"_1", "W_"+label+"_2"), requires_grad = True) \
#                              ).to(device)
#         self.V = Variable(torch.randn((layers,self.domain.columns), names = ("V_"+label+"_0", "V_"+label+"_1"), requires_grad = True)).to(device)
#         self.b = Variable(torch.neg(torch.ones(1,layers,   names = ("b_"+label+"_0","b_"+label+"_1"), requires_grad = True))).to(device)
#         self.u = Variable(torch.ones((layers,1),    names = ("u_"+label+"_0","u_"+label+"_1"), requires_grad = True)).to(device)
        self.W = Variable(torch.randn((layers, self.domain.columns, self.domain.columns)).to(device) , requires_grad = True )
        self.V = Variable(torch.randn((layers,self.domain.columns)).to(device), requires_grad = True)
        self.b = Variable(torch.neg(torch.ones(1,layers).to(device)), requires_grad = True)
        self.u = Variable(torch.ones(layers,1).to(device), requires_grad = True)
        self.parameters = [self.W,self.V,self.b,self.u]

    def tensor(self,domain=None):
        if domain is None:
            domain = self.domain
        X = domain.tensor
#         print(X.shape)
        XW = torch.bmm(torch.tile(torch.unsqueeze(X, 0), [self.number_of_layers, 1,1]), self.W)
#         print("XW: ", XW.shape)
#         print(torch.unsqueeze(X, 1).shape,  torch.permute(XW, [1, 2, 0]).shape)
        XWX = (torch.bmm(torch.unsqueeze(X, 1), torch.permute(XW, [1, 2, 0]))).squeeze(1)
#         print("XWX: ", XWX.shape)
        XV = torch.matmul(X, torch.permute(self.V,[1,0]))
        gX = torch.matmul(torch.tanh(XWX + XV + self.b),self.u)
        return torch.sigmoid(gX)
    
class Literal:
    def __init__(self,polarity,predicate,domain=None):
        self.predicate = predicate
        self.polarity = polarity
        if domain is None:
            self.domain = predicate.domain
        else:
            self.domain = domain
        if polarity:
            self.tensor = predicate.tensor(domain)
        else:
            if default_tnorm == "product" or default_tnorm == "goedel":
                xx = predicate.tensor(domain)
                y = xx==(torch.zeros(xx.shape).to(device))
                self.tensor = y.float().to(dtype=torch.float32)
            if default_tnorm == "yager2":
                self.tensor = 1-predicate.tensor(domain)
            if default_tnorm == "luk":
                self.tensor = 1-predicate.tensor(domain)

        self.parameters = predicate.parameters + domain.parameters

def disjunction_of_literals(literals,label="no_label"):
    list_of_literal_tensors = [lit.tensor for lit in literals]
    literals_tensor = torch.concat(list_of_literal_tensors,1)
    if default_tnorm == "product":
        result = 1.0-torch.prod(1.0-literals_tensor,1,keepdim=True)
    if default_tnorm=="yager2":
        result = torch.minimum(1.0,torch.sqrt(torch.sum(torch.square(literals_tensor),1, keepdim=True)))
    if default_tnorm=="luk":
        result = torch.minimum(1.0,torch.sum(literals_tensor,1, keepdim=True))
    if default_tnorm == "goedel":
        result = torch.max(literals_tensor,1,keepdim=True)
    if default_aggregator == "product":
        return torch.prod(result,keepdim=True)
    if default_aggregator == "mean":
        return torch.mean(result,keepdim=True)
    if default_aggregator == "gmean":
        return torch.exp(torch.mul(torch.sum(torch.log(result), keepdim=True),
                             torch.inv(torch.to_float(torch.size(result)))))
    if default_aggregator == "hmean":
        return torch.div(torch.to_float(torch.size(result)),torch.sum(torch.inv(result),keepdim=True))
    if default_aggregator == "min":
        return torch.min(result)
       
class Clause:
    def __init__(self,literals,label=None, weight=1.0):
        self.weight=weight
        self.label=label
        self.literals = literals
        self.tensor = disjunction_of_literals(literals,label=label)
        self.predicates = set([lit.predicate for lit in self.literals])
        self.parameters = [par for lit in literals for par in lit.parameters]
        

parts_of_whole, wholes_of_part = get_part_whole_ontology()

w1 = {}
p1 = {}
w2 = {}
p2 = {}
p1w1 = {}
p2w2 = {}
oo = Domain((number_of_features-1)*2+2, label="same_object_pairs")
o = Domain(number_of_features-1, label="a_generi_object")

w0 = Domain(number_of_features - 1, label="whole_of_part_whole_pair")
p0 = Domain(number_of_features - 1, label="part_of_part_whole_pair")
p0w0 = Domain((number_of_features - 1) * 2 + 2, label="part_whole_pair")
w0p0 = Domain((number_of_features - 1) * 2 + 2, label="whole_part_pair")  
objects = Domain(number_of_features-1,label="a_bounding_box")
pairs_of_objects = Domain(2*(number_of_features-1)+2,label="a_pair_of_bounding_boxes")

for t in selected_types:
    w1[t] = Domain(number_of_features-1, label="whole_predicted_objects_for_"+id2types[t])
    p1[t] = Domain(number_of_features-1, label="part_predicted_objects_for_"+id2types[t])
    w2[t] = Domain(number_of_features - 1, label="whole_predicted_objects_for_" + id2types[t])
    p2[t] = Domain(number_of_features - 1, label="part_predicted_objects_for_" + id2types[t])
    p1w1[t] = Domain((number_of_features-1)*2+2, label="potential_part_whole_object_pairs_for_"+id2types[t])
    p2w2[t] = Domain((number_of_features-1)*2+2, label="potential_whole_part_object_pairs_for_"+id2types[t])




objects_of_type = {}
objects_of_type_not = {}
for t in selected_types:
    objects_of_type[t] = Domain(number_of_features-1,label="objects_of_type_"+id2types[t])
    objects_of_type_not[t] = Domain(number_of_features-1,label="objects_of_type_not_" + id2types[t])

object_pairs_in_partOf = Domain((number_of_features-1) * 2 + 2,
                                    label="object_pairs_in_partof_relation")
object_pairs_not_in_partOf = Domain((number_of_features-1) * 2 + 2,
                                        label="object_pairs_not_in_partof_relation")



############################################
def add_noise_to_data(noise_ratio):
    if noise_ratio > 0:
        freq_other = {}

        for t in selected_types:
            freq_other[t] = {}
            number_of_not_t = len(idxs_of_negative_examples_of_types[t])
            for t1 in selected_types:
                if t1 != t:
                    freq_other[t][t1] = np.float(len(idxs_of_positive_examples_of_types[t1]))/number_of_not_t

        noisy_data_idxs = np.random.choice(range(len(data)), int(len(data) * noise_ratio),replace=False)

        for idx in noisy_data_idxs:
            type_of_idx = types_of_data[idx]
            not_types_of_idx = np.setdiff1d(selected_types,type_of_idx)
            types_of_data[idx] = np.random.choice(not_types_of_idx,
                                                        p=np.array([freq_other[type_of_idx][t1] \
                                                                    for t1 in not_types_of_idx]))

        noisy_data_pairs_idxs = np.append(np.random.choice(np.where(partOf_of_pairs_of_data)[0],
                                                 int(partOf_of_pairs_of_data.sum() * noise_ratio * 0.5)),
                                          np.random.choice(np.where(np.logical_not(partOf_of_pairs_of_data))[0],
                                                           int(partOf_of_pairs_of_data.sum() * noise_ratio* 0.5)))

        for idx in noisy_data_pairs_idxs:
            partOf_of_pairs_of_data[idx] = not (partOf_of_pairs_of_data[idx])

    idxs_of_noisy_positive_examples_of_types = {}
    idxs_of_noisy_negative_examples_of_types = {}

    for type in selected_types:
        idxs_of_noisy_positive_examples_of_types[type] = np.where(types_of_data == type)[0]
        idxs_of_noisy_negative_examples_of_types[type] = np.where(types_of_data != type)[0]

    idxs_of_noisy_positive_examples_of_partOf = np.where(partOf_of_pairs_of_data)[0]
    idxs_of_noisy_negative_examples_of_partOf = np.where(partOf_of_pairs_of_data == False)[0]

    print("I have introduces the followins errors")
    for t in selected_types:
        print("wrong positive", t, len(np.setdiff1d(idxs_of_noisy_positive_examples_of_types[t],
                                                    idxs_of_positive_examples_of_types[t])))
        print("wrong negative", t, len(np.setdiff1d(idxs_of_noisy_negative_examples_of_types[t],
                                                    idxs_of_negative_examples_of_types[t])))

    print( "wrong positive partof", len(np.setdiff1d(idxs_of_noisy_positive_examples_of_partOf,
                                                    idxs_of_positive_examples_of_partOf)))
    print("wrong negative partof", len(np.setdiff1d(idxs_of_noisy_negative_examples_of_partOf,
                                                    idxs_of_negative_examples_of_partOf)))

    return idxs_of_noisy_positive_examples_of_types,idxs_of_noisy_negative_examples_of_types,  idxs_of_noisy_positive_examples_of_partOf,  idxs_of_noisy_negative_examples_of_partOf


idxs_of_pos_ex_of_types,idxs_of_neg_ex_of_types,  idxs_of_pos_ex_of_partOf,  idxs_of_neg_ex_of_partOf = add_noise_to_data(0.2)

for t in existing_types:
    print(idxs_of_pos_ex_of_types[t], number_of_positive_examples_x_types)
    objects_of_type[t].tensor =  torch.Tensor(data[np.random.choice(idxs_of_pos_ex_of_types[t],   number_of_positive_examples_x_types)][:,1:]).to(device)
    objects_of_type_not[t].tensor = torch.Tensor(data[np.random.choice(idxs_of_neg_ex_of_types[t],  number_of_negative_examples_x_types)][:, 1:]).to(device)

    # positive and negative examples for partOF
    object_pairs_in_partOf.tensor =    torch.Tensor(pairs_of_data[np.random.choice(idxs_of_pos_ex_of_partOf, number_of_positive_example_x_partof)]).to(device)

    object_pairs_not_in_partOf.tensor =   torch.Tensor(pairs_of_data[np.random.choice(idxs_of_neg_ex_of_partOf,  number_of_negative_example_x_partof)]).to(device)

    # feed data for axioms
    tmp = pairs_of_data[np.random.choice(range(pairs_of_data.shape[0]), number_of_pairs_for_axioms)]
    o.tensor =torch.Tensor(tmp[:,:number_of_features - 1]).to(device)

   
    for t in selected_types:
        p1w1[t].tensor = torch.Tensor(tmp).to(device)
        w1[t].tensor = p1w1[t].tensor[:,number_of_features-1:2*(number_of_features-1)].to(device)
        p1[t].tensor = p1w1[t].tensor[:,0:number_of_features - 1].to(device)
        p2w2[t].tensor = torch.Tensor(tmp).to(device)
        w2[t].tensor = p2w2[t].tensor[:, number_of_features - 1:2 * (number_of_features - 1)].to(device)
        p2[t].tensor = p2w2[t].tensor[:,:number_of_features - 1].to(device)

    oo.tensor = torch.Tensor(np.concatenate([tmp[:,:number_of_features-1],  tmp[:,:number_of_features-1],   np.ones((tmp.shape[0],2),dtype=float)],axis=1)).to(device)
    p0w0.tensor = tmp
    w0.tensor =  torch.Tensor(p0w0.tensor[:, number_of_features - 1:2 * (number_of_features - 1)]).to(device)
    p0.tensor =  torch.Tensor(p0w0.tensor[:,:number_of_features - 1]).to(device)
    xxx = torch.Tensor(tmp[:, -1:-3:-1].copy()).to(device)
    w0p0.tensor = torch.concat([
        w0.tensor,
        p0.tensor,
        xxx], axis = 1).to(device)
    p0w0.tensor = torch.Tensor(tmp).to(device)
    
        
        
def smooth(parameters):
    norm_of_omega = torch.sum(torch.concat([torch.sum(torch.square(par)).unsqueeze(0) for par in parameters],0).unsqueeze(1))
    return torch.mul(default_smooth_factor,norm_of_omega)

# def PR(tensor):
#     global count
#     torch.set_printoptions()
#     return torch.Print(tensor,[tensor.shape,tensor.name,tensor],summarize=200000)

        
class LTNModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.isOfType = {}
        self.isPartOf = Predicate("is_part_of",pairs_of_objects).to(device)
        self.parameters = self.isPartOf.parameters
        for t in selected_types:
            self.isOfType[t] = Predicate("is_of_type_"+id2types[t],objects,layers=5).to(device)
            self.parameters += self.isOfType[t].parameters
        
    def penalize_positive_facts(self):
        tensor_for_positive_facts = [torch.sum(Literal(True,lit.predicate,lit.domain).tensor) for cl in self.clauses for lit in cl.literals]
        return torch.sum(torch.stack(tensor_for_positive_facts))
        
    def forward(self):
           
        clauses_for_positive_examples_of_types = [Clause([Literal(True, self.isOfType[t],objects_of_type[t])],label="examples_of_"+id2types[t],weight=1.0) for t in existing_types]

        clauses_for_negative_examples_of_types =  [Clause([Literal(False,self.isOfType[t],objects_of_type_not[t])],label="examples_of_not_"+id2types[t],weight=1.0) for t in existing_types]

        clause_for_positive_examples_of_partOf = [Clause([Literal(True, self.isPartOf, object_pairs_in_partOf)], label="examples_of_object_pairs_in_partof_relation", weight=1.0)]

        clause_for_negative_examples_of_partOf = [Clause([Literal(False, self.isPartOf, object_pairs_not_in_partOf)], label="examples_of_object_pairs_not_in_part_of_relation", weight=1.0)]

         # defining the clauses of the background knowledge
        clauses = clauses_for_positive_examples_of_types +  clauses_for_negative_examples_of_types +   clause_for_positive_examples_of_partOf +   clause_for_negative_examples_of_partOf

        partOf_is_antisymmetric = [Clause([Literal(False, self.isPartOf, p0w0), Literal(False, self.isPartOf, w0p0)], label="part_of_is_antisymmetric", weight=0.37)]

        partof_is_irreflexive = [Clause([Literal(False, self.isPartOf, oo)],label = "part_of_is_irreflexive", weight = 0.37)]

        clauses_for_parts_of_wholes = [Clause([Literal(False, self.isOfType[w], w1[w]),   Literal(False, self.isPartOf, p1w1[w])] +    [Literal(True, self.isOfType[p], p1[w]) for p in parts_of_whole[w]], label = "parts_of_" + id2types[w]) for w in parts_of_whole.keys()]

        clauses_for_wholes_of_parts = [Clause([Literal(False, self.isOfType[p], p2[p]),   Literal(False, self.isPartOf, p2w2[p])] +  [Literal(True, self.isOfType[w], w2[p]) for w in wholes_of_part[p]], label="wholes_of_" + id2types[p]) for p in wholes_of_part.keys()]

        clauses_for_disjoint_types = [Clause([Literal(False,self.isOfType[t],o),  Literal(False,self.isOfType[t1],o)],label=id2types[t]+"_is_not_"+id2types[t1]) for t in selected_types for t1 in selected_types if t < t1]

        clause_for_at_least_one_type = [Clause([Literal(True,self.isOfType[t],o) for t in selected_types], label="an_object_has_at_least_one_type")]
        
        clauses += partof_is_irreflexive +  partOf_is_antisymmetric +  clauses_for_wholes_of_parts +   clauses_for_parts_of_wholes +   clauses_for_disjoint_types +  clause_for_at_least_one_type
#         print(clauses.tensor)
        self.clauses = clauses
        clauses_value_tensor = torch.stack( [cl.tensor for cl in clauses])
        if default_clauses_aggregator == "min":
            print =("clauses aggregator is min")
            self.tensor = torch.min(clauses_value_tensor)
        if default_clauses_aggregator == "mean":
            self.tensor = torch.mean(clauses_value_tensor)
        if default_clauses_aggregator == "hmean":
            self.tensor = torch.div((clauses_value_tensor).shape.float(), torch.sum(clauses_value_tensor.T, keepdim=True))
        if default_clauses_aggregator == "wmean":
            weights_tensor = torch.constant([cl.weight for cl in clauses])
            self.tensor = torch.div(torch.sum(torch.mul(weights_tensor, clauses_value_tensor)),torch.sum(weights_tensor))
        self.loss = smooth(self.parameters) + default_positive_fact_penality* self.penalize_positive_facts() -self.tensor#-   PR(self.tensor)
#         print("loss: ", str(self.loss))
        return self.loss

/tmp/ipykernel_736275/1087952319.py:189: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  freq_other[t][t1] = np.float(len(idxs_of_positive_examples_of_types[t1]))/number_of_not_t


I have introduces the followins errors
wrong positive 24 184
wrong negative 24 153
wrong positive 18 233
wrong negative 18 272
wrong positive 43 110
wrong negative 43 96
wrong positive 32 132
wrong negative 32 138
wrong positive 42 147
wrong negative 42 128
wrong positive 36 113
wrong negative 36 135
wrong positive 2 116
wrong negative 2 113
wrong positive 28 100
wrong negative 28 100
wrong positive partof 306
wrong negative partof 278
[   2   11   15   19   21   23   29   40   47   72   82   83   90   97
  119  132  133  145  148  149  155  157  178  188  203  204  209  216
  225  227  230  232  235  238  244  250  252  254  256  258  260  261
  265  266  268  270  272  274  276  278  280  282  284  286  291  292
  297  300  303  307  322  334  339  341  342  344  358  362  364  366
  369  383  385  387  404  417  426  428  440  443  446  449  452  455
  458  469  471  473  502  515  534  555  561  564  567  571  574  585
  588  594  603  605  612  617  668  670  680  682  684  686  6

[  24   37   51   54   57   60   63   69   75   78   81   87  107  110
  113  116  127  164  168  171  174  177  191  193  195  200  213  220
  249  293  311  313  316  330  331  347  349  352  355  376  379  391
  394  397  400  403  406  409  412  419  430  463  466  476  479  484
  509  518  521  524  527  533  539  542  548  551  554  568  595  631
  639  646  650  655  658  673  719  722  744  747  750  762  799  800
  826  841  844  867  870  891  906  916  940  943  946  951  959  960
 1001 1008 1011 1014 1017 1021 1024 1029 1055 1058 1072 1086 1123 1156
 1157 1160 1163 1166 1169 1182 1185 1188 1191 1196 1225 1228 1231 1234
 1237 1239 1242 1245 1251 1269 1310 1319 1332 1345 1347 1348 1365 1390
 1393 1396 1400 1411 1414 1424 1441 1449 1458 1464 1467 1468 1472 1476
 1482 1490 1534 1541 1627 1632 1641 1646 1679 1682 1685 1693 1696 1699
 1702 1720 1726 1729 1732 1740 1741 1743 1744 1780 1797 1804 1816 1840
 1844 1847 1858 1864 1872 1878 1883 1895 1898 1901 1938 1947 1950 1972
 1978 

In [220]:
import ltn
# by default, SatAgg uses the pMeanError
model = LTNModel().to(device)
# we need to learn the parameters of the predicate C
optimizer = torch.optim.RMSprop(model.parameters, lr=0.001)

for epoch in range(1000):
    optimizer.zero_grad()
    loss = model()
    loss.backward()
    optimizer.step()
    if epoch%10 == 0:
        print("Epoch %d: Sat Level %.3f "%(epoch, 1 - loss.item()))
print("Training finished at Epoch %d with Sat Level %.3f" %(epoch, 1 - loss.item()))

Epoch 0: Sat Level 0.947 
Epoch 10: Sat Level 0.959 
Epoch 20: Sat Level 0.963 
Epoch 30: Sat Level 0.965 
Epoch 40: Sat Level 0.967 
Epoch 50: Sat Level 0.968 
Epoch 60: Sat Level 0.970 
Epoch 70: Sat Level 0.970 
Epoch 80: Sat Level 0.971 
Epoch 90: Sat Level 0.972 
Epoch 100: Sat Level 0.973 
Epoch 110: Sat Level 0.973 
Epoch 120: Sat Level 0.974 
Epoch 130: Sat Level 0.974 
Epoch 140: Sat Level 0.975 
Epoch 150: Sat Level 0.975 
Epoch 160: Sat Level 0.976 
Epoch 170: Sat Level 0.976 
Epoch 180: Sat Level 0.977 
Epoch 190: Sat Level 0.977 
Epoch 200: Sat Level 0.978 
Epoch 210: Sat Level 0.978 
Epoch 220: Sat Level 0.978 
Epoch 230: Sat Level 0.979 
Epoch 240: Sat Level 0.979 
Epoch 250: Sat Level 0.980 
Epoch 260: Sat Level 0.980 
Epoch 270: Sat Level 0.980 
Epoch 280: Sat Level 0.981 
Epoch 290: Sat Level 0.981 
Epoch 300: Sat Level 0.981 
Epoch 310: Sat Level 0.981 
Epoch 320: Sat Level 0.982 
Epoch 330: Sat Level 0.982 
Epoch 340: Sat Level 0.982 
Epoch 350: Sat Level 0.983 
Epo

KeyboardInterrupt: 

In [260]:
data, pairs_of_data, types_of_data, partOf_of_pairs_of_data, pairs_of_bb_idxs, pics = get_data("train",max_rows=1000000000)
objects.tensor = torch.Tensor(data[:,1:]).to(device)
pairs_of_objects.tensor = torch.Tensor(pairs_of_data).to(device)
predicted_types_values_tensor = torch.concat([model.isOfType[t].tensor(objects_of_type[t]) for t in existing_types],1).detach().cpu().numpy()
predicted_partOf_value_tensor = Literal(True,model.isPartOf,pairs_of_objects).tensor
# values_of_types = predicted_types_values_tensor,{objects.tensor:data[:,1:]}
# values_of_partOf = predicted_partOf_value_tensor,{pairs_of_objects.tensor:pairs_of_data}


deleting 140540 small bb out of 146217 bb


In [243]:

# predicted_partOf_value_tensor = (predicted_partOf_value_tensor).squeeze().detach().cpu().numpy()
# predicted_types_values_tensor.shape
types = [t  for t,w in types.items() if w in selected_types]

In [267]:
import matplotlib.pyplot as plt

confusion_matrix = np.zeros((len(selected_types) + 1, len(selected_types) + 1))
decision_threshold = 0.7
for t,ii in enumerate(selected_types):
    if t not in idxs_of_pos_ex_of_types: continue
    for idx_bb in idxs_of_pos_ex_of_types[ii]:
        predicted_label_idx = np.argmax(predicted_types_values_tensor[idx_bb,:])
        if predicted_types_values_tensor[idx_bb,predicted_label_idx] >= decision_threshold:
            confusion_matrix[t, predicted_label_idx] += 1
        else:
            confusion_matrix[t, len(selected_types)] += 1 # assign to background class

fig = plt.figure(figsize=(10.0, 10.0))
cm_normalized = confusion_matrix.astype('float') / confusion_matrix.sum(axis=1)[:, np.newaxis]
plot_confusion_matrix(cm_normalized, types + ['bkg'], types + ['bkg'], title='CM for types: ')
#plt.show() #uncomment for show confusion matrix

fp = confusion_matrix.sum(axis=0)
fn = confusion_matrix.sum(axis=1)
prec = np.true_divide(np.diag(confusion_matrix),fp)[0:-1] # remove background class
rec = np.true_divide(np.diag(confusion_matrix),fn)[0:-1] # remove background class
f1 = np.true_divide(2*prec*rec, prec + rec)

mean_prec = np.mean(prec)
mean_rec = np.mean(rec)
mean_f1 = np.mean(f1)

parts, wholes = get_part_whole_ontology()
part_labels = wholes.keys()
prec_wholes = []
rec_wholes = []
f1_wholes = []
prec_parts = []
rec_parts = []
f1_parts = []

for t in types:
    if t in part_labels:
        prec_parts.append(prec[types.index(t)])
        rec_parts.append(rec[types.index(t)])
        f1_parts.append(f1[types.index(t)])
    else:
        prec_wholes.append(prec[types.index(t)])
        rec_wholes.append(rec[types.index(t)])
        f1_wholes.append(f1[types.index(t)])

mean_prec_wholes = np.mean(prec_wholes)
mean_rec_wholes = np.mean(rec_wholes)
mean_f1_wholes = np.mean(f1_wholes)
mean_prec_parts = np.mean(prec_parts)
mean_rec_parts = np.mean(rec_parts)
mean_f1_parts = np.mean(f1_parts)

prec_rec_f1 = np.concatenate(([number_training_examples], [prec], [rec], [f1]), axis=0)

if not os.path.exists(save_path):
    os.makedirs(save_path)

np.savetxt(os.path.join(save_path, 'CM_types_' + label + '.csv'), confusion_matrix, delimiter=",")
fig.savefig(os.path.join(save_path, 'CM_types_' + label + '.png'))

print("th %f --> prec %f, rec %f, f1 %f" % (decision_threshold, mean_prec, mean_rec, mean_f1))


IndexError: index 287 is out of bounds for axis 0 with size 250

In [274]:
np.max(predicted_types_values_tensor[111])

9.594977e-05

In [269]:
objects_

NameError: name 'objects_of_test' is not defined

In [215]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support
p_ltn = torch.argmax(predicted_types_values_tensor.detach().cpu(), 1)
h_ltn = np.array([existing_types[p] for p in p_ltn])
y_ltn = []
y_true = []
for idx,t in enumerate(types_of_data):
    if t in existing_types:
        y_ltn.append(selected_types_name[p_ltn[idx]])
        y_true.append(selected_types_name[np.where(existing_types==t)[0][0]])
        print(y_ltn[idx], y_true[idx])

# confusion_matrix(y_true, y_pred, labels=selected_types_name)
p,r,f,s = precision_recall_fscore_support(y_true, y_ltn)
print(precision_recall_fscore_support(y_true, y_ltn,average='macro'))
print({selected_types_name[i]:s[i] for i in range(len(selected_types))})
print(f)

bottle tvmonitor
cap screen
pottedplant bottle
pottedplant body
body bottle
body body
bottle tvmonitor
cap screen
cap body
bottle tvmonitor
cap screen
cap bottle
tvmonitor cap
cap body
plant body
pot bottle
cap cap
pot body
cap body
body bottle
body body
body bottle
body body
plant bottle
plant body
cap body
cap tvmonitor
cap screen
bottle tvmonitor
pottedplant bottle
pottedplant body
cap body
pot bottle
cap cap
pot body
screen pottedplant
screen plant
screen pot
bottle tvmonitor
bottle screen
cap bottle
cap body
cap tvmonitor
cap screen
pot body
plant body
pot body
body bottle
pot body
pot pottedplant
pot plant
cap pot
cap pottedplant
pot plant
pottedplant pot
pottedplant pottedplant
pottedplant plant
pottedplant pot
cap pottedplant
plant plant
cap pot
pot pottedplant
pot plant
body pot
pot pottedplant
pot plant
plant pot
pottedplant pottedplant
pottedplant plant
pottedplant pot
bottle pottedplant
bottle plant
bottle pot
cap pottedplant
pottedplant plant
cap pot
tvmonitor pottedplant


(['bottle',
  'cap',
  'pottedplant',
  'pottedplant',
  'body',
  'body',
  'bottle',
  'cap',
  'cap',
  'bottle',
  'cap',
  'cap',
  'tvmonitor',
  'cap',
  'plant',
  'pot',
  'cap',
  'pot',
  'cap',
  'body',
  'body',
  'body',
  'body',
  'plant',
  'plant',
  'cap',
  'cap',
  'cap',
  'bottle',
  'pottedplant',
  'pottedplant',
  'cap',
  'pot',
  'cap',
  'pot',
  'screen',
  'screen',
  'screen',
  'bottle',
  'bottle',
  'cap',
  'cap',
  'cap',
  'cap',
  'pot',
  'plant',
  'pot',
  'body',
  'pot',
  'pot',
  'pot',
  'cap',
  'cap',
  'pot',
  'pottedplant',
  'pottedplant',
  'pottedplant',
  'pottedplant',
  'cap',
  'plant',
  'cap',
  'pot',
  'pot',
  'body',
  'pot',
  'pot',
  'plant',
  'pottedplant',
  'pottedplant',
  'pottedplant',
  'bottle',
  'bottle',
  'bottle',
  'cap',
  'pottedplant',
  'cap',
  'tvmonitor',
  'cap',
  'cap',
  'screen',
  'screen',
  'cap',
  'cap',
  'pottedplant',
  'cap',
  'cap',
  'pot',
  'cap',
  'bottle',
  'cap',
  'cap',


In [171]:
import numpy as np
import torch
from collections import Counter

number_of_test_data_per_type = Counter(types_of_data)
print(number_of_test_data_per_type)
type_cardinality_array = np.array([number_of_test_data_per_type[t] for t in selected_types])
idxs_for_selected_types = selected_types#np.concatenate([np.where(types == st)[0] for st in selected_types])
print(idxs_for_selected_types)

thresholds = np.arange(.00,1.1,.05)
# computing confusion matrixes for the prediction of a model
def confusion_matrixes_of_model(values_of_types, values_of_partOf, thresholds):
    print("computing confusion matrix for", path_to_model)
    global data, types_of_data, partOf_of_pairs_of__data
    print(values_of_types.shape, values_of_partOf.shape)
    confusion_matrix_for_types = {}
    confusion_matrix_for_pof = {}
    #pdb.set_trace()
    for th in thresholds:
        print(th," ")
        confusion_matrix_for_types[th] = np.matrix([[0.0] * len(selected_types)] * len(selected_types))
        for bb_idx in range(len(data)):
            for st_idx in range(len(selected_types)):
                if values_of_types[bb_idx][st_idx] >= th:
#                     print(st_idx, np.where(selected_types == types_of_data[bb_idx])[0][0], values_of_types[typeid2cid[selected_types[st_idx]]][bb_idx])
                    confusion_matrix_for_types[th][st_idx, np.where(selected_types == types_of_data[bb_idx])[0][0]] += 1
        confusion_matrix_for_pof[th] = np.matrix([[0.0, 0.0], [0.0, 0.0]])
        for bb_pair_idx in range(len(pairs_of_data)):
            if values_of_partOf[bb_pair_idx] >= th:
                if partOf_of_pairs_of_data[bb_pair_idx]:
                    confusion_matrix_for_pof[th][0, 0] += 1
                else:
                    confusion_matrix_for_pof[th][0, 1] += 1
            else:
                if partOf_of_pairs_of_data[bb_pair_idx]:
                    confusion_matrix_for_pof[th][1, 0] += 1
                else:
                    confusion_matrix_for_pof[th][1, 1] += 1

    return confusion_matrix_for_types, confusion_matrix_for_pof

def precision(conf_matrix, prediction_array=None):
    if prediction_array is not None:
        return conf_matrix.diagonal()/prediction_array
    else:
        return conf_matrix.diagonal() / conf_matrix.sum(1).T

def recall(conf_matrix,gold_array=None):
    if gold_array is not None:
        return conf_matrix.diagonal() / gold_array
    else:
        return conf_matrix.diagonal() / conf_matrix.sum(0)

def f1(precision,recall):
    return np.multiply(2*precision,recall)/(precision + recall)

measure_per_type = {}
measure_per_pof = {}

measures = ["prec","recall","f1"]

for measure in measures:
    measure_per_pof[measure] = {}
    measure_per_type[measure] = {}

paths_to_models = ["dfalc"]
for path_to_model in paths_to_models:
    if path_to_model == "baseline":
        cm_types, cm_pof = confusion_matrixes_of_model(cEmb_candid.T.detach().numpy(), rEmb_candid[0].detach().numpy(), thresholds)
    else:
        cm_types, cm_pof = confusion_matrixes_of_model(predicted_types_values_tensor.detach().cpu().numpy(), predicted_partOf_value_tensor,thresholds)
        print(cm_types)
    for measure in measures:
        measure_per_type[measure][path_to_model] = {}
        measure_per_pof[measure][path_to_model] = {}
    for th in thresholds:
        measure_per_type["prec"][path_to_model][th] = precision(cm_types[th])
        measure_per_type["recall"][path_to_model][th] = recall(cm_types[th],gold_array=type_cardinality_array)
        measure_per_type["f1"][path_to_model][th] = f1(measure_per_type["prec"][path_to_model][th],
                                                        measure_per_type["recall"][path_to_model][th])
        measure_per_pof["prec"][path_to_model][th] = precision(cm_pof[th])
        measure_per_pof["recall"][path_to_model][th] = recall(cm_pof[th])
        measure_per_pof["f1"][path_to_model][th] = f1(measure_per_pof["prec"][path_to_model][th],
                                                       measure_per_pof["recall"][path_to_model][th])
    

Counter({18: 1376, 24: 825, 32: 663, 42: 648, 36: 603, 2: 557, 28: 506, 43: 499})
[24, 18, 43, 32, 42, 36, 2, 28]
computing confusion matrix for dfalc
(5677, 8) (51761,)
0.0  
0.05  
0.1  
0.15000000000000002  
0.2  
0.25  
0.30000000000000004  
0.35000000000000003  
0.4  
0.45  
0.5  
0.55  
0.6000000000000001  
0.65  
0.7000000000000001  
0.75  
0.8  
0.8500000000000001  
0.9  
0.9500000000000001  
1.0  
1.05  
{0.0: matrix([[ 825., 1376.,  499.,  663.,  648.,  603.,  557.,  506.],
        [ 825., 1376.,  499.,  663.,  648.,  603.,  557.,  506.],
        [ 825., 1376.,  499.,  663.,  648.,  603.,  557.,  506.],
        [ 825., 1376.,  499.,  663.,  648.,  603.,  557.,  506.],
        [ 825., 1376.,  499.,  663.,  648.,  603.,  557.,  506.],
        [ 825., 1376.,  499.,  663.,  648.,  603.,  557.,  506.],
        [ 825., 1376.,  499.,  663.,  648.,  603.,  557.,  506.],
        [ 825., 1376.,  499.,  663.,  648.,  603.,  557.,  506.]]), 0.05: matrix([[0., 0., 0., 0., 0., 0., 0., 0.],

/tmp/ipykernel_736275/3485415662.py:47: RuntimeWarning: invalid value encountered in true_divide
  return conf_matrix.diagonal() / conf_matrix.sum(1).T


In [172]:
measure_per_type

{'prec': {'dfalc': {0.0: matrix([[0.14532323, 0.24238154, 0.08789854, 0.11678704, 0.11414479,
            0.10621807, 0.0981152 , 0.08913158]]),
   0.05: matrix([[nan, nan, nan, nan, nan, nan, nan, nan]]),
   0.1: matrix([[nan, nan, nan, nan, nan, nan, nan, nan]]),
   0.15000000000000002: matrix([[nan, nan, nan, nan, nan, nan, nan, nan]]),
   0.2: matrix([[nan, nan, nan, nan, nan, nan, nan, nan]]),
   0.25: matrix([[nan, nan, nan, nan, nan, nan, nan, nan]]),
   0.30000000000000004: matrix([[nan, nan, nan, nan, nan, nan, nan, nan]]),
   0.35000000000000003: matrix([[nan, nan, nan, nan, nan, nan, nan, nan]]),
   0.4: matrix([[nan, nan, nan, nan, nan, nan, nan, nan]]),
   0.45: matrix([[nan, nan, nan, nan, nan, nan, nan, nan]]),
   0.5: matrix([[nan, nan, nan, nan, nan, nan, nan, nan]]),
   0.55: matrix([[nan, nan, nan, nan, nan, nan, nan, nan]]),
   0.6000000000000001: matrix([[nan, nan, nan, nan, nan, nan, nan, nan]]),
   0.65: matrix([[nan, nan, nan, nan, nan, nan, nan, nan]]),
   0.70

In [36]:
for param in model.isOfType: #查看可优化的参数有哪些
    print(param.requires_grad)

AttributeError: 'dict' object has no attribute 'parameters'

In [53]:
model.isOfType[54].tensor(data[:,1:])

AttributeError: 'numpy.ndarray' object has no attribute 'tensor'

In [40]:
model.isOfType[54].parameters[0].requires_grad

False